# f) Ngonye Falls Power and Energy Periods
Produces summary, agrregated results from the daily power and energy modelling.


## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| [Model Name]_pe_daily.csv | Notebook: e_power_energy | Daily power output  |


## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|
|    [Model Name]_pe_daily.csv ||
|    [Model Name]_pe_monthly.csv||
|    [Model Name]_pe_yearly.csv||
|    [Model Name]_pe_percentiles.csv||
|    [Model Name]_pe_monthly_day.csv||
|    [Model Name]_pe_daily_slim.csv||
|    [Model Name]_pe_weekly.csv||
|    [Model Name]_pe_weekly_slim.csv||
|    [Model Name]_pe_calmonthly.csv||
|    [Model Name]_pe_fdc.csv||
|    [Model Name]_pe_power_exceed.csv||
|    models.csv| |

## Libraries and Setup

In [29]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math

## Parameters

In [30]:
input_data='./input_data/'
output_data='./output_data/'

In [2]:
model_name='Base Case'#MW153#MW144 'MW162'#'Fixed Tailwater'#'Base Case' Headpond50cm EWRAllCs
dryrun=False #Dont write output files

In [32]:
models=pd.read_csv(input_data + "models.csv").set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'
if out_prefix!='base':
    out_prefix="scenarios/" + out_prefix
    
plant_capacity=model['Capacity']#180
canal_capacity=plant_capacity/180*1100#1100

model

OutputPrefix                                  fixedtail
Description       Tailwater level remains fixed at 970m
EWRCatSet                              Recommendation 1
HeadpondLift                                          0
Capacity                                            180
FixedTailwater                                      970
Mean                                             831429
P95                                              628030
P90                                              681508
P75                                              777449
P50                                              837483
P25                                              919293
P10                                              988011
P5                                          1.01648e+06
StDev                                            125170
CoefVar                                        0.150549
Max                                         1.06441e+06
Min                                             

## Load Daily Data

Load the daily time series which includes the exceedance values used for calculating daily EWRs.

In [33]:
daily=pd.read_csv(output_data+out_prefix+'_pe_daily.csv')
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)
daily.head(5)

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,...,EffGen2,EffGen3,EffGen4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec
Date,,,,,,,,,,,,,,,,,,,,,
1924-10-01,1924-10-12,100.0,1.111331,111.133124,0.999,1924,10,1,1924.1,1924,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1924-10-02,1924-10-13,100.0,1.111331,111.133124,0.999,1924,10,2,1924.1,1924,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1924-10-03,1924-10-14,100.0,1.111331,111.133124,0.999,1924,10,3,1924.1,1924,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1924-10-04,1924-10-15,100.0,1.111331,111.133124,0.999,1924,10,4,1924.1,1924,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1924-10-05,1924-10-16,100.0,1.111331,111.133124,0.999,1924,10,5,1924.1,1924,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


## Monthly


In [34]:

monthly=daily.groupby("MonthId").mean().drop(['VicFalls','Conversion','Day','WaterDay','WaterWeek','Volume','EWRBandNo','EWRRefExceedance','Energy'],axis=1)

monthly=monthly.join(daily.groupby("MonthId").agg(    
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),   
   Turbines_max=('Turbines', 'max'),    
   Turbines_min=('Turbines', 'min'),   
   Energy=('Energy', 'sum'),
))
monthly['Days']=monthly.apply(lambda x: pd.Period(year=x['Year'],month=x['Month'],freq='D').days_in_month,axis=1)
monthly['CapFactor']=monthly['Energy']/(plant_capacity*24*monthly['Days'])

monthly.tail(12)

,Flow,Exceedance,Year,Month,WaterYear,WaterMonth,EWRChannelA,EWRChannelC,EWRChannelD,EWRChannelE,...,PowerTurbine4,Power,PerfMwPerCumec,Power_max,Power_min,Turbines_max,Turbines_min,Energy,Days,CapFactor
MonthId,,,,,,,,,,,,,,,,,,,,,
2016.10,186.117272,0.973290,2016.0,10.0,2016.0,1.0,31.000000,4.000000,3.000000,2.000000,...,NaN,19.042767,0.101933,23.361011,15.067903,1,1,14167.818899,31,0.105793
2016.11,186.562065,0.968767,2016.0,11.0,2016.0,2.0,25.000000,4.000000,2.000000,2.000000,...,NaN,22.287206,0.117795,31.406827,14.364816,1,1,16046.788557,30,0.123818
2016.12,369.487694,0.701516,2016.0,12.0,2016.0,3.0,29.387097,4.677419,3.000000,2.000000,...,NaN,51.839992,0.139418,88.700281,31.215012,2,1,38568.954287,31,0.288000
2017.01,851.896141,0.394581,2017.0,1.0,2016.0,4.0,44.387097,6.677419,5.354839,3.677419,...,38.758405,122.858670,0.141765,161.639210,54.665794,4,2,91406.850805,31,0.682548
2017.02,1236.957587,0.273679,2017.0,2.0,2016.0,5.0,49.000000,8.000000,7.000000,5.000000,...,42.369247,169.476989,0.138126,180.292753,155.511663,4,4,113888.536705,28,0.941539
2017.03,2420.771572,0.145774,2017.0,3.0,2016.0,6.0,49.000000,9.000000,9.000000,9.000000,...,44.470190,177.880761,0.082246,177.988599,177.869089,4,4,132343.286385,31,0.988226
2017.04,3419.680887,0.063633,2017.0,4.0,2016.0,7.0,70.000000,14.000000,18.000000,25.000000,...,44.467272,177.869089,0.052540,177.869089,177.869089,4,4,128065.744189,30,0.988162
2017.05,2194.251076,0.156387,2017.0,5.0,2016.0,8.0,55.000000,13.000000,13.000000,17.000000,...,44.467272,177.869089,0.082900,177.869089,177.869089,4,4,132334.602328,31,0.988162
2017.06,1117.935234,0.312067,2017.0,6.0,2016.0,9.0,53.000000,12.000000,12.000000,13.000000,...,42.271424,144.236187,0.129391,178.985676,80.894847,4,2,103850.054503,30,0.801312


## Yearly

In [35]:
yearly=daily.groupby("WaterYear").agg(
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
)
yearly['CapFactor']=yearly['Energy']/(plant_capacity*24*365)


In [36]:
yearly['Prop4Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==4,1,0))/365)
yearly['Prop3Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==3,1,0))/365)
yearly['Prop2Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==2,1,0))/365)
yearly['Prop1Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==1,1,0))/365)
yearly['Prop0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0))/365)
yearly['Days0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0)))


## Decades

## Power and energy exceedance

In [37]:
power_exceed=pd.DataFrame(index=np.arange(0,1.05,0.05))
power_exceed.index=power_exceed.index.rename('Exceedance')
power_exceed['Power']=np.percentile(daily['Power'],(1-power_exceed.index)*100)
power_exceed.head(3)

,Power
Exceedance,
0.00,185.715522
0.05,178.030221
0.10,177.869089


In [38]:
edc=pd.DataFrame(index=np.arange(0,1.05,0.05))
edc['Energy']=np.percentile(yearly['Energy'],(1-edc.index)*100)

edc.head(3)

,Energy
0.00,1.064406e+06
0.05,1.016482e+06
0.10,9.880112e+05


In [39]:
yearly['Exceedance']=pd.merge_asof(yearly.reset_index().sort_values('Energy'),edc.reset_index().sort_values('Energy'),left_on='Energy',right_on='Energy').set_index('WaterYear')['index']

## Calendar years
Jan-Dec rather than Oct-Sept

In [40]:
calyearly=daily.groupby("Year").agg(  
   Energy=('Energy', 'sum'),
).drop([1924,2017],axis=0)
calyearly['CapFactor']=calyearly['Energy']/(180*24*365)
calyearly.head(5)

,Energy,CapFactor
Year,,
1925,839242.925265,0.532244
1926,790095.350801,0.501075
1927,833028.761979,0.528303
1928,782265.032253,0.496109
1929,654074.756833,0.414811


## Monthly Day
15th of every month

In [41]:
monthly_day=daily.loc[daily['Day']==15]

## Daily Slim
Slim has less columns and values are rounded so file size is reduced

In [42]:
daily_slim=daily.loc[:,
                     ['Year',
                      'Month',
                      'Day',
                      'WaterYear',
                      'Flow',
                      'EWRRefExceedance',
                      'EWRBandNo',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'Turbines',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',    
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion': 2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,    
                    })
daily_slim.tail(12)

,Year,Month,Day,WaterYear,Flow,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,EffTurbine1,EffTurbine2,EffTurbine3,EffTurbine4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-19,2017,9,19,2016,216,0.81,4,34,6,4,...,0.91,0.0,0.0,0.0,21.5,0.0,0.0,0.0,21.5,515
2017-09-20,2017,9,20,2016,213,0.81,4,34,6,4,...,0.91,0.0,0.0,0.0,20.9,0.0,0.0,0.0,20.9,502
2017-09-21,2017,9,21,2016,213,0.82,4,34,6,4,...,0.91,0.0,0.0,0.0,20.9,0.0,0.0,0.0,20.9,502
2017-09-22,2017,9,22,2016,209,0.82,4,34,6,4,...,0.90,0.0,0.0,0.0,20.1,0.0,0.0,0.0,20.1,482
2017-09-23,2017,9,23,2016,203,0.82,4,34,6,4,...,0.90,0.0,0.0,0.0,18.8,0.0,0.0,0.0,18.8,452
2017-09-24,2017,9,24,2016,203,0.82,4,34,6,4,...,0.90,0.0,0.0,0.0,18.8,0.0,0.0,0.0,18.8,451
2017-09-25,2017,9,25,2016,203,0.82,4,34,6,4,...,0.90,0.0,0.0,0.0,18.8,0.0,0.0,0.0,18.8,451
2017-09-26,2017,9,26,2016,203,0.82,4,34,6,4,...,0.90,0.0,0.0,0.0,18.8,0.0,0.0,0.0,18.8,451
2017-09-27,2017,9,27,2016,203,0.82,4,34,6,4,...,0.90,0.0,0.0,0.0,18.8,0.0,0.0,0.0,18.8,451


## Weekly & Weekly Slim
Slim has less columns and values are rounded so file size is reduced

In [43]:
weekly=daily.groupby(["WaterYear","WaterWeek"]).mean().drop(['Month','Year','MonthId','WaterMonth','VicFalls','Conversion','Day','WaterDay','Volume','EWRBandNo','EWRRefExceedance','Turbines','Energy'],axis=1)
weekly=weekly.join(daily.groupby(["WaterYear","WaterWeek"]).agg(    
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
))
weekly=weekly.join(daily[["WaterYear","WaterWeek","Flow"]].groupby(["WaterYear","WaterWeek"]).count().rename(columns={'Flow':'Count'}))
weekly['CapFactor']=weekly['Energy']/(180*24*weekly['Count'])
weekly=weekly.join(daily.reset_index('Date')[["WaterYear","WaterWeek","Date"]].groupby(["WaterYear","WaterWeek"]).first().rename(columns={'Date':'StartDate'}))

weekly.tail(12)

Flow  Exceedance  EWRChannelA  EWRChannelC  \
WaterYear WaterWeek                                                     
2016      42         451.673869    0.594143    50.000000    10.000000   
          43         411.631943    0.638429    50.000000    10.000000   
          44         377.984000    0.677571    43.714286     8.285714   
          45         345.089425    0.718000    39.000000     7.000000   
          46         322.320457    0.753571    39.000000     7.000000   
          47         296.353596    0.789429    39.000000     7.000000   
          48         272.698436    0.833000    38.285714     6.857143   
          49         250.364895    0.874143    34.000000     6.000000   
          50         228.116819    0.912143    34.000000     6.000000   
          51         215.508577    0.932000    34.000000     6.000000   
          52         203.039212    0.949714    34.000000     6.000000   
          53         198.229576    0.958000    34.000000     6.000000   

                     EWRChannelD  EWRChannelE  EWRChannelFG    EWRTotal  \
WaterYear WaterWeek                                                       
2016      42            9.000000     9.000000    108.000000  186.000000   
          43            9.000000     9.000000    108.000000  186.000000   
          44            6.714286     6.142857     80.571429  145.428571   
          45            5.000000     4.000000     60.000000  115.000000   
          46            5.000000     4.000000     60.000000  115.000000   
          47            5.000000     4.000000     60.000000  115.000000   
          48            4.857143     3.857143     57.714286  111.571429   
          49            4.000000     3.000000     44.000000   91.000000   
          50            4.000000     3.000000     44.000000   91.000000   
          51            4.000000     3.000000     44.000000   91.000000   
          52            4.000000     3.000000     44.000000   91.000000   
          53            4.000000     3.000000     44.000000   91.000000   

                     EWRProportion   FlowCanal  ...  HeadTurbine1_max  \
WaterYear WaterWeek                             ...                     
2016      42              0.412191  265.673869  ...         19.684353   
          43              0.452114  225.631943  ...         19.528773   
          44              0.383130  232.555429  ...         19.589478   
          45              0.333428  230.089425  ...         19.517312   
          46              0.357011  207.320457  ...         19.600791   
          47              0.388246  181.353596  ...         19.667126   
          48              0.408941  161.127008  ...         19.727443   
          49              0.363667  159.364895  ...         19.730750   
          50              0.399211  137.116819  ...         19.780522   
          51              0.422374  124.508577  ...         19.803742   
          52              0.448189  112.039212  ...         19.817218   
          53              0.459064  107.229576  ...         19.826805   

                     HeadTurbine1_min  LevelTailwater_max  LevelTailwater_min  \
WaterYear WaterWeek                                                             
2016      42                19.331390                 970                 970   
          43                19.423033                 970                 970   
          44                19.344565                 970                 970   
          45                19.436074                 970                 970   
          46                19.525296                 970                 970   
          47                19.607766                 970                 970   
          48                19.671137                 970                 970   
          49                19.676359                 970                 970   
          50                19.742607                 970                 970   
          51                19.780522  

In [44]:
weekly_slim=weekly.loc[:,
                     [
                      'StartDate',
                      'Flow',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion':2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,
                    })

## Calendar Months

In [45]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly=calmonthly.join(monthly.groupby(["WaterMonth"]).agg(    
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_mean=('EWRProportion', 'mean'),
   EWRProportion_median=('EWRProportion', 'median'),
   EWRProportion_min=('EWRProportion', 'min'),
    
    EWRTotal_max=('EWRTotal', 'max'),
   EWRTotal_mean=('EWRTotal', 'mean'),
   EWRTotal_median=('EWRTotal', 'median'),
   EWRTotal_min=('EWRTotal', 'min'),   
    
   FlowCanal_max=('FlowCanal', 'max'),
   FlowCanal_mean=('FlowCanal', 'mean'),
   FlowCanal_median=('FlowCanal', 'median'),
   FlowCanal_min=('FlowCanal', 'min'),     
    
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_mean=('HeadTurbine1', 'mean'),
   HeadTurbine1_median=('HeadTurbine1', 'median'),
   HeadTurbine1_min=('HeadTurbine1', 'min'), 

   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_mean=('LevelTailwater', 'mean'),    
   LevelTailwater_median=('LevelTailwater', 'median'),    
   LevelTailwater_min=('LevelTailwater', 'min'),  
   LevelHeadpond_max=('LevelHeadpond', 'max'),
   LevelHeadpond_mean=('LevelHeadpond', 'mean'),    
   LevelHeadpond_median=('LevelHeadpond', 'median'),    
   LevelHeadpond_min=('LevelHeadpond', 'min'),  
   Turbines_mean=('Turbines', 'mean'),      
   Power_max=('Power', 'max'),
   Power_mean=('Power', 'mean'),
   Power_median=('Power', 'median'),
   Power_min=('Power', 'min'),    
   Energy_max=('Energy', 'max'),
   Energy_mean=('Energy', 'mean'),
   Energy_median=('Energy', 'median'),    
   Energy_min=('Energy', 'min'),
))
calmonthly['Energy_P95']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.05)
calmonthly['Energy_P90']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.10)
calmonthly['Energy_P75']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.25)
calmonthly['Energy_P25']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.75)
calmonthly['Energy_P10']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.90)
calmonthly['Energy_P5']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.95)

calmonthly['Energy_stdev']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').std()
calmonthly['Energy_coefvar']=calmonthly['Energy_stdev']/calmonthly['Energy_mean']

calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,Energy_median,Energy_min,Energy_P95,Energy_P90,Energy_P75,Energy_P25,Energy_P10,Energy_P5,Energy_stdev,Energy_coefvar
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1,Oct,10,0.580145,0.397072,0.399437,0.274238,120.0,93.623309,97.000000,52.0,...,16844.914114,5189.468741,13116.978632,14179.080626,14945.307991,21136.859980,24830.849749,29149.967945,5698.584381,0.308426
2,Nov,11,0.403580,0.331341,0.335545,0.219146,114.0,89.311828,93.000000,50.0,...,21258.520552,12385.611075,15298.714381,16916.425358,18189.476802,25605.033692,32841.683761,36683.435778,7403.307402,0.318866
3,Dec,12,0.288862,0.242016,0.242502,0.158090,126.0,100.352064,104.000000,59.0,...,38568.954287,22000.142417,28991.706360,30440.616029,34296.886076,49773.775889,56731.134288,71610.055324,12878.343955,0.301707
4,Jan,1,0.222615,0.177630,0.181580,0.097584,150.0,117.763441,120.000000,73.0,...,64398.006092,33284.744592,45702.384270,50672.306639,57248.249113,84501.566574,107508.095743,114523.196292,21977.012743,0.303295
5,Feb,2,0.188613,0.130120,0.140135,0.023962,197.0,139.328407,137.000000,85.0,...,94788.874324,43324.828486,62353.369720,67911.544749,80958.393390,113888.536705,119536.580953,122652.417760,20641.535852,0.217941
6,Mar,3,0.183460,0.095537,0.091553,0.040247,315.0,194.510926,185.451613,107.0,...,132334.602328,62049.933665,94612.431593,100577.014235,122637.773962,132340.226335,132450.898888,132801.318219,14582.403044,0.117189
7,Apr,4,0.165319,0.085341,0.083222,0.049988,364.0,231.824373,217.000000,125.0,...,128065.744189,78609.897631,116238.627709,126412.278212,128065.744189,128073.491437,128219.037766,128449.602233,8339.955672,0.066197
8,May,5,0.223318,0.130912,0.128444,0.066398,340.0,265.271245,276.000000,147.0,...,132334.602328,69194.883084,91757.709636,119225.247100,132334.602328,132338.585252,132393.301027,132440.677964,13967.040983,0.109541
9,Jun,6,0.369589,0.200097,0.194879,0.108743,293.0,212.774194,210.000000,119.0,...,108133.725537,25986.054545,41164.349577,60670.994723,84002.052417,119753.367852,126956.782261,128010.630456,26832.372507,0.268980


## Flow duration curve

In [ ]:
daily['ExceedanceRnd']=np.round(daily['Exceedance'],2)
fdc=daily.reset_index().groupby('ExceedanceRnd').mean().drop(['VicFalls','Conversion','Year','Month','Day','MonthId','WaterYear','WaterDay','WaterMonth','WaterWeek'],axis=1)
fdc

## Add overall summary data to the model file

In [49]:
models.at[model_name,'Mean']=yearly['Energy'].mean()
models.at[model_name,'Max']=yearly['Energy'].max()
models.at[model_name,'Min']=yearly['Energy'].min()

models.at[model_name,'P95']=yearly['Energy'].quantile(0.05)
models.at[model_name,'P90']=yearly['Energy'].quantile(0.10)
models.at[model_name,'P75']=yearly['Energy'].quantile(0.25)
models.at[model_name,'P50']=yearly['Energy'].quantile(0.5)
models.at[model_name,'P25']=yearly['Energy'].quantile(0.75)
models.at[model_name,'P10']=yearly['Energy'].quantile(0.90)
models.at[model_name,'P5']=yearly['Energy'].quantile(0.95)

models.at[model_name,'StDev']=yearly['Energy'].std()
models.at[model_name,'CoefVar']=models.at[model_name,'StDev']/models.at[model_name,'Mean']

models.at[model_name,'CapFactor']=yearly['CapFactor'].mean()

models.at[model_name,'CalYearMean']=calyearly['Energy'].mean()
models.at[model_name,'CalYearStDev']=calyearly['Energy'].std()
models.at[model_name,'CalYearCoefVar']=models.at[model_name,'CalYearStDev']/models.at[model_name,'CalYearMean']

models.loc[model_name]



,OutputPrefix,Description,EWRCatSet,HeadpondLift,Capacity,FixedTailwater,Mean,P95,P90,P75,...,P10,P5,StDev,CoefVar,Max,Min,CapFactor,CalYearMean,CalYearStDev,CalYearCoefVar
ModelName,,,,,,,,,,,,,,,,,,,,,
Base Case,base,Base Case,Recommendation 1,0.0,180,0,832122.545498,717411.25064,754939.090685,791546.596541,...,921509.942677,9.577369e+05,82094.797365,0.098657,1.018276e+06,493500.384743,0.527729,832218.371546,78428.056566,0.094240
Fixed Tailwater,fixedtail,Tailwater level remains fixed at 970m,Recommendation 1,0.0,180,970,831428.890433,628030.20447,681507.597416,777448.679781,...,988011.212459,1.016482e+06,125170.399893,0.150549,1.064406e+06,405736.964143,0.527289,831459.422541,128603.459685,0.154672
MW171,mw171,Capacity reduced to 171MW,Recommendation 1,0.0,171,0,821322.470600,710283.28930,749082.584200,784712.402100,...,905799.600000,9.412670e+05,78909.883820,0.096077,1.001265e+06,493939.272200,0.548294,821466.855100,75593.892410,0.092023
MW162,mw162,Capacity reduced to 162MW,Recommendation 1,0.0,162,0,807544.251500,700369.41080,743105.679700,771463.932000,...,887605.100800,9.219910e+05,75470.247140,0.093456,9.821224e+05,494262.823100,0.569046,807733.563900,72182.233080,0.089364
MW153,mw153,Capacity reduced to 153MW,Recommendation 1,0.0,153,0,788302.161600,690333.33500,725082.533500,756159.366500,...,861285.706400,8.973242e+05,71059.284050,0.090142,9.552291e+05,495335.578000,0.588162,788555.791300,67709.474480,0.085865
MW144,mw144,Capacity reduced to 142MW,Recommendation 1,0.0,142,0,766570.714100,679744.38590,708282.128200,737361.910300,...,839574.466200,8.708112e+05,66555.613010,0.086823,9.255126e+05,496517.701200,0.616254,766929.035800,63119.329870,0.082301
Headpond50cm,head50,Headpond level increased by 50cm,Recommendation 1,0.5,180,0,858051.924400,737056.66560,779147.891800,815574.348700,...,947605.155700,9.841974e+05,84985.169370,0.099044,1.045190e+06,504075.011300,0.544173,858170.224700,82120.183390,0.095692
EWRAllCs,allcs,All river reaches at a C rather than B environ...,All Cs,0.0,180,0,860068.256900,744670.63170,784275.896400,823016.545700,...,950632.482700,9.861146e+05,81610.662760,0.094889,1.047438e+06,525711.094800,0.545452,860230.075500,77888.569300,0.090544


## Save Files

In [47]:
if dryrun==False:
    daily.to_csv(output_data+out_prefix+'_pe_daily.csv')
    monthly.to_csv(output_data+out_prefix+'_pe_monthly.csv')
    yearly.to_csv(output_data+out_prefix+'_pe_yearly.csv')
    edc.to_csv(output_data+out_prefix+'_pe_percentiles.csv')
    monthly_day.to_csv(output_data+out_prefix+'_pe_monthly_day.csv')
    daily_slim.to_csv(output_data+out_prefix+'_pe_daily_slim.csv')
    weekly.to_csv(output_data+out_prefix+'_pe_weekly.csv')
    weekly_slim.to_csv(output_data+out_prefix+'_pe_weekly_slim.csv')
    calmonthly.to_csv(output_data+out_prefix+'_pe_calmonthly.csv')
    fdc.to_csv(output_data+out_prefix+'_pe_fdc.csv')
    power_exceed.to_csv(output_data+out_prefix+'_pe_power_exceed.csv')
    models.to_csv(input_data + "models.csv")